# N+2 token train

In [ ]:
## CIFAR 10 Model Analysis
# from numpy.random import choice

# set_seed()

# VALID_CLASSES = ["ALL"]

# transform = transforms.Compose(
#     [
#         transforms.PILToTensor(),
#         transforms.ConvertImageDtype(torch.float),
#     ]
# )

# dataset = CIFAR10(".", download=True, transform=transform)

# mask_indices = [idx for idx, s in enumerate(dataset) if s[1] in VALID_CLASSES]
# mask_indices = choice(mask_indices, 1000, replace=False)
# dataset = Subset(dataset, mask_indices)
# # train_size = int(0.02 * len(dataset))
# train_size = 1
# BATCH_SIZE = 1

# train_ds, x_ds = random_split(dataset, [train_size, len(dataset) - train_size])

# print("Number of samples in dataset: ", len(train_ds))

# del dataset
# del x_ds

# train_dl = DataLoader(train_ds, BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
# PRETRAINED_MODEL_NAME = "aaraki/vit-base-patch16-224-in21k-finetuned-cifar10"  # Pretrained on ImageNet, FineTuned on CIFAR 10
# FEAT_EXTRACT = ViTImageProcessor()
# VIT_MODEL = to_device(
#     ViTForImageClassification.from_pretrained(PRETRAINED_MODEL_NAME), device
# )

# def get_pretrained_vit_outputs(batch: tuple):
#     with torch.no_grad():

#         ip_imgs = batch[0]
#         ip_labels = batch[1]

#         inputs = to_device(
#             FEAT_EXTRACT.preprocess(
#                 images=ip_imgs, data_format="channels_first", return_tensors="pt"
#             )["pixel_values"],
#             device,
#         )
#         outputs = VIT_MODEL(inputs, output_attentions=True, output_hidden_states=True)
#         preds = outputs.logits.argmax(dim=1)

#         return {"hidden_states": outputs.hidden_states}

# all_images_data = {}
# # Processing for [START_LAYER, END_LAYER)
# START_LAYER = 0
# END_LAYER = 13
# NUM_LAYERS = END_LAYER - START_LAYER
# TOTAL_LAYERS = 13 # +1 for starting embedding dimension
#     # for batch in train_dl:
#     #     ops = get_pretrained_vit_outputs(batch)

#     #     for idx in range(START_LAYER, END_LAYER):
#     #         all_images_data[label]["hidden_states"][idx] = torch.cat(
#     #             [
#     #                 all_images_data[label]["hidden_states"][idx],
#     #                 ops["hidden_states"][idx].detach().cpu(),
#     #             ]
#     #         )

#     # print("Processing for label complete: ", label)

# def get_layer_variance(t_var: torch.Tensor) -> torch.Tensor:
#     # (batch_size B, seq_len S, emb_dim D)
#     if t_var.shape[0] != 1:
#         t_var = torch.var(t_var, dim=(0,), keepdim=True)

#     # Remove CLS
#     t_var = t_var[0, 1:, :]

#     # Mean along D
#     t_var = torch.mean(t_var, dim=(1,))
#     t_var = einops.rearrange(t_var, "(h w) -> h w", h=14, w=14)
#     t_var = einops.repeat(t_var, "x y -> (x p1) (y p2)", x=14, y=14, p1=16, p2=16)

#     return t_var
# for label in VALID_CLASSES:
#     # class_labels = torch.tensor([label])
#     print(f"Processing for Label: {label} for layers {START_LAYER} to {END_LAYER}")

#     all_images_data[label] = {
#         # "hidden_states": [torch.empty(0, 197, 768) for _ in range(TOTAL_LAYERS)],
#         "variance": [None for _ in range(NUM_LAYERS)],
#     }

#     for layer_idx in range(START_LAYER, END_LAYER):
#         # RAM crash so redundancy
#         layer_variance = torch.empty(0, 197, 768)

#         for batch in train_dl:
#             layer_hidden_state = get_pretrained_vit_outputs(batch)["hidden_states"][layer_idx].detach().cpu()
#             layer_variance = torch.cat([layer_variance, layer_hidden_state])

#         layer_variance = get_layer_variance(layer_variance)

#         all_images_data[label]["variance"][layer_idx] = layer_variance
# from torchvision.transforms.functional import resize

# # 1000 images for all classes
# import matplotlib as mpl

# mpl.rcParams["figure.dpi"] = 100

# t_var = all_images_data[label]["variance"][2]
# img = resize(train_ds[0][0], size=(224, 224), antialias=True)
# plt.imshow(img.permute(1, 2, 0))
# plt.imshow(t_var, cmap="gray", alpha=0.2)
# # # One class at a time

# # fig, axs = plt.subplots(NUM_LAYERS, figsize=(10, 15))
# # fig.suptitle(f"Variance along seq for class: {VALID_CLASSES}")
# # fig.tight_layout()
# # class_label = VALID_CLASSES[0]

# # for idx, state in enumerate(
# #     all_images_data[class_label]["hidden_states"][START_LAYER:END_LAYER]
# # ):
# #     # (B, S, D) -> (S)
# #     op = torch.var(state, dim=(0, 2))

# #     axs[idx].plot(op)
# #     axs[idx].set_title(f"Layer: {idx + START_LAYER}", fontsize=10, loc="right")

# # plt.show()
# # fig, axs = plt.subplots(NUM_LAYERS, figsize=(10, 15))
# # fig.suptitle(f"Variance along patch for class: {VALID_CLASSES}")
# # fig.tight_layout()
# # class_label = VALID_CLASSES[0]

# # for idx, state in enumerate(
# #     all_images_data[class_label]["hidden_states"][START_LAYER:END_LAYER]
# # ):
# #     # (B, S, D) -> (S)
# #     op = torch.var(state, dim=(0, 1))

# #     axs[idx].plot(op)
# #     axs[idx].set_title(f"Layer: {idx + START_LAYER}", fontsize=10, loc="right")

# # plt.show()

# # 1000 images for all classes
# import matplotlib as mpl

# mpl.rcParams["figure.dpi"] = 400

# fig, axs = plt.subplots(NUM_LAYERS, figsize=(16, 20), sharex=True)
# title = f"ViT - Variance Patch wise 1 image CIFAR-10 (224 x 224)"
# fig.tight_layout()

# img = resize(train_ds[0][0], size=(224, 224), antialias=True).permute(1, 2, 0)

# for idx, state in enumerate(all_images_data[VALID_CLASSES[0]]["variance"]):
#     print("Processing for Layer: ", idx)
#     # Normalized Variance
#     # tensor_variance = 20 * torch.log10(1 + state)
#     tensor_variance = state

#     axs[idx].set_title(f"Depth: {START_LAYER + idx}", fontsize=10, loc="right")
#     axs[idx].xaxis.set_visible(False)
#     axs[idx].yaxis.set_visible(False)
#     axs[idx].imshow(img, alpha=1.0)

#     im = axs[idx].imshow(tensor_variance, cmap="jet", alpha=0.5)
#     plt.colorbar(im, ax=axs[idx])


# axs[5].set_title(title, rotation="vertical", x=-0.1, y=0.5, fontsize=10)

# plt.show()